In [12]:
""" usage:
   python3 wd_search.py <string> [<required types>]
   python3 Adobe
   python3 Adobe company
"""

from wd_search import wd_search, prettyPrint

In [15]:
def test(string, types=[]):
    print('searching for', string, types)
    prettyPrint(wd_search(string, types))

Examples to try 
 * test('acrobat')
 * test('ukraine')
 * test('anonymous')
 * test('ibm')
 * test('black hat')
 * test('Shadow Brokers')
 * test('julian assange')
 * test('ddos')
 * test('linux')
 * get_types('Q27134643')

In [17]:
test('ddos')

searching for ddos []
[ { 'aliases': ['DDoS'],
    'concepturi': 'http://www.wikidata.org/entity/Q17329819',
    'description': 'cyber attack',
    'id': 'Q17329819',
    'label': 'distributed denial-of-service attack',
    'match': {'language': 'en', 'text': 'DDoS', 'type': 'alias'},
    'types': [('Q4071928', 'cyberattack')]}]
